## 概述

Transformer是一种神经网络结构，由Vaswani等人在2017年的论文“Attention Is All You Need”中提出，用于处理机器翻译、语言建模和文本生成等自然语言处理任务。

与带有Attention机制的Seq2Seq模型所不同的是，Transformer模型是一个纯基于注意力机制的架构，不包含任何循环神经网络（Recurrent Neural Network， RNN）或卷积神经网络（Convolutional Neural Network，CNN）。其中，在处理输入序列上，Transformer引入了自注意力机制（self-attention mechanism）。使其能够衡量输入序列中不同部分的重要性，并有选择性地关注最相关的信息，从而能够更有效地捕捉数据中的长期依赖关系。

Transformer的另一个关键特征是它在对输入序列进行编码时， 在原有Embedding上添加了位置编码（Positional Encoding），弥补了自注意力机制中缺失的位置信息，使模型在训练中可以学习这部分信息，区分具有相同数字序列单不同位置的单词。

总体而言，与NLP领域中的前序模型（LSTM、GRU、Seq2Seq等）相比，Transformer具有几个优点：
1. 更容易并行化，训练更加高效；
2. 运算中不会改变输入数据的shape，大大减少了搭建模型中统一tensor shape的工作；
3. 在处理长序列的任务中表现优秀，可以快速捕捉长距离中的关联信息。

上述优势使其成为各种自然语言处理任务的热门选择，并在其架构基础上衍生出了BERT、GPT等模型。

## 数据准备

我们本次使用的数据集为**Multi30K数据集**，它是一个大规模的图像-文本数据集，包含30K+图片，每张图片对应两类不同的文本描述：
- 英语描述，及对应的德语翻译；
- 五个独立的、非翻译而来的英语和德语描述，描述中包含的细节并不相同；

因其收集的不同语言对于图片的描述相互独立，所以训练出的模型可以更好地适用于有噪声的多模态内容。

![avatar](./assets/Multi30K.png)
> 图片来源：Elliott, D., Frank, S., Sima’an, K., & Specia, L. (2016). Multi30K: Multilingual English-German Image Descriptions. CoRR, 1605.00459.

首先，我们需要下载如下依赖：

- 分词工具：`pip install spacy`
- 德语/英语分词器：`python -m spacy download de_core_news_sm`，`python -m spacy download en_core_web_sm`

### 数据下载模块

使用`download`进行数据下载，并将`tar.gz`文件解压到指定文件夹。

下载好的数据集目录结构如下：

```text
home_path/.mindspore_examples
├─test
│      test2016.de
│      test2016.en
│      test2016.fr
│
├─train
│      train.de
│      train.en
│
└─valid
        val.de
        val.en
```

In [2]:
from download import download
from pathlib import Path
from tqdm import tqdm
import os

# 训练、验证、测试数据集下载地址
urls = {
    'train': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz',
    'valid': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz',
    'test': 'http://www.quest.dcs.shef.ac.uk/wmt17_files_mmt/mmt_task1_test2016.tar.gz'
}

# 指定保存路径为 `home_path/.mindspore_examples`
cache_dir = Path.home() / '.mindspore_examples'

train_path = download(urls['train'], os.path.join(cache_dir, 'train'), kind='tar.gz')
valid_path = download(urls['valid'], os.path.join(cache_dir, 'valid'), kind='tar.gz')
test_path = download(urls['test'], os.path.join(cache_dir, 'test'), kind='tar.gz')

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.


### 数据预处理

在使用数据进行模型训练等操作时，我们需要对数据进行预处理，流程如下：

1. 加载数据集，目前数据为句子形式的文本，需要进行分词，即将句子拆解为单独的词元（token，可以为字符或者单词）；
    - 分词可以使用`spaCy`创建分词器（tokenizer）：`de_core_news_sm`，`en_core_web_sm`，需要手动下载；
    - 分词后，去除多余的空格，统一大小写等；
2. 将每个词元映射到从0开始的数字索引中（为节约存储空间，可过滤掉词频低的词元），词元和数字索引所构成的集合叫做词典（vocabulary）；
3. 添加特殊占位符，标明序列的起始与结束，统一序列长度，并创建数据迭代器；

#### 数据加载器

In [3]:
import spacy
from functools import partial

class Multi30K():
    """Multi30K数据集加载器

    加载Multi30K数据集并处理为一个Python迭代对象。

    """
    def __init__(self, path):
        self.data = self._load(path)

    def _load(self, path):
        def tokenize(text, spacy_lang):
            # 去除多余空格，统一大小写
            text = text.rstrip()
            return [tok.text.lower() for tok in spacy_lang.tokenizer(text)]

        # 加载英、德语分词器
        tokenize_de = partial(tokenize, spacy_lang=spacy.load('de_core_news_sm'))
        tokenize_en = partial(tokenize, spacy_lang=spacy.load('en_core_web_sm'))

        # 读取Multi30K数据，并进行分词
        members = {i.split('.')[-1]: i for i in os.listdir(path)}
        de_path = os.path.join(path, members['de'])
        en_path = os.path.join(path, members['en'])
        with open(de_path, 'r') as de_file:
            de = de_file.readlines()[:-1]
            de = [tokenize_de(i) for i in de]
        with open(en_path, 'r') as en_file:
            en = en_file.readlines()[:-1]
            en = [tokenize_en(i) for i in en]

        return list(zip(de, en))

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

In [4]:
train_dataset, valid_dataset, test_dataset = Multi30K(train_path), Multi30K(valid_path), Multi30K(test_path)

对解压和分词结果进行测试，打印测试数据集第一组英德语文本，可以看到每一个单词和标点符号已经被单独分离出来。

In [5]:
for de, en in test_dataset:
    print(f'de = {de}')
    print(f'en = {en}')
    break

de = ['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.']
en = ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']


#### 词典

In [6]:
class Vocab:
    """通过词频字典，构建词典"""

    special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']

    def __init__(self, word_count_dict, min_freq=1):
        self.word2idx = {}
        for idx, tok in enumerate(self.special_tokens):
            self.word2idx[tok] = idx

        # 过滤低词频的词元
        filted_dict = {
            w: c
            for w, c in word_count_dict.items() if c >= min_freq
        }
        for w, _ in filted_dict.items():
            self.word2idx[w] = len(self.word2idx)

        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        self.bos_idx = self.word2idx['<bos>']  # 特殊占位符：序列开始
        self.eos_idx = self.word2idx['<eos>']  # 特殊占位符：序列结束
        self.pad_idx = self.word2idx['<pad>']  # 特殊占位符：补充字符
        self.unk_idx = self.word2idx['<unk>']  # 特殊占位符：低词频词元或未曾出现的词元

    def _word2idx(self, word):
        """单词映射至数字索引"""
        if word not in self.word2idx:
            return self.unk_idx
        return self.word2idx[word]

    def _idx2word(self, idx):
        """数字索引映射至单词"""
        if idx not in self.idx2word:
            raise ValueError('input index is not in vocabulary.')
        return self.idx2word[idx]

    def encode(self, word_or_list):
        """将单个单词或单词数组映射至单个数字索引或数字索引数组"""
        if isinstance(word_or_list, list):
            return [self._word2idx(i) for i in word_or_list]
        return self._word2idx(word_or_list)

    def decode(self, idx_or_list):
        """将单个数字索引或数字索引数组映射至单个单词或单词数组"""
        if isinstance(idx_or_list, list):
            return [self._idx2word(i) for i in idx_or_list]
        return self._idx2word(idx_or_list)

    def __len__(self):
        return len(self.word2idx)

通过自定义词频字典进行测试，我们可以看到词典已去除词频少于2的词元c，并加入了默认的四个特殊占位符，故词典整体长度为：4 - 1 + 4 = 7

In [7]:
word_count = {'a':20, 'b':10, 'c':1, 'd':2}

vocab = Vocab(word_count, min_freq=2)
len(vocab)

7

使用`collections`中的`Counter`和`OrderedDict`统计英/德语每个单词在整体文本中出现的频率。构建词频字典，然后再将词频字典转为词典。

在分配数字索引时有一个小技巧：常用的词元对应数值较小的索引，这样可以节约空间。

In [8]:
from collections import Counter, OrderedDict

def build_vocab(dataset):
    de_words, en_words = [], []
    for de, en in dataset:
        de_words.extend(de)
        en_words.extend(en)

    de_count_dict = OrderedDict(sorted(Counter(de_words).items(), key=lambda t: t[1], reverse=True))
    en_count_dict = OrderedDict(sorted(Counter(en_words).items(), key=lambda t: t[1], reverse=True))

    return Vocab(de_count_dict, min_freq=2), Vocab(en_count_dict, min_freq=2)

In [9]:
de_vocab, en_vocab = build_vocab(train_dataset)
print('Unique tokens in de vocabulary:', len(de_vocab))

Unique tokens in de vocabulary: 7853


#### 数据迭代器

数据预处理的最后一步是创建数据迭代器，我们在进一步处理数据（包括批处理，添加起始和终止符号，统一序列长度）后，将数据以张量的形式返回。

创建数据迭代器需要如下参数：

- `dataset`：分词后的数据集
- `de_vocab`：德语词典
- `en_vocab`：英语词典
- `batch_size`：批量大小，即一个batch中包含多少个序列
- `max_len`：序列最大长度，为最长有效文本长度 + 2（序列开始、序列结束占位符），如不满则补齐，如超过则丢弃
- `drop_remainder`：是否在最后一个batch未满时，丢弃该batch

In [10]:
import mindspore

class Iterator():
    """创建数据迭代器"""
    def __init__(self, dataset, de_vocab, en_vocab, batch_size, max_len=32, drop_reminder=False):
        self.dataset = dataset
        self.de_vocab = de_vocab
        self.en_vocab = en_vocab

        self.batch_size = batch_size
        self.max_len = max_len
        self.drop_reminder = drop_reminder

        length = len(self.dataset) // batch_size
        self.len = length if drop_reminder else length + 1  # 批量数量

    def __call__(self):
        def pad(idx_list, vocab, max_len):
            """统一序列长度，并记录有效长度"""
            idx_pad_list, idx_len = [], []
            # 当前序列度超过最大长度时，将超出的部分丢弃；当前序列长度小于最大长度时，用占位符补齐
            for i in idx_list:
                if len(i) > max_len - 2:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i[:max_len-2] + [vocab.eos_idx]
                    )
                    idx_len.append(max_len)
                else:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i + [vocab.eos_idx] + [vocab.pad_idx] * (max_len - len(i) - 2)
                    )
                    idx_len.append(len(i) + 2)
            return idx_pad_list, idx_len

        def sort_by_length(src, trg):
            """对德/英语的字段长度进行排序"""
            data = zip(src, trg)
            data = sorted(data, key=lambda t: len(t[0]), reverse=True)
            return zip(*list(data))

        def encode_and_pad(batch_data, max_len):
            """将批量中的文本数据转换为数字索引，并统一每个序列的长度"""
            # 将当前批量数据中的词元转化为索引
            src_data, trg_data = zip(*batch_data)
            src_idx = [self.de_vocab.encode(i) for i in src_data]
            trg_idx = [self.en_vocab.encode(i) for i in trg_data]

            # 统一序列长度
            src_idx, trg_idx = sort_by_length(src_idx, trg_idx)
            src_idx_pad, src_len = pad(src_idx, de_vocab, max_len)
            trg_idx_pad, _ = pad(trg_idx, en_vocab, max_len)

            return src_idx_pad, src_len, trg_idx_pad

        for i in range(self.len):
            # 获取当前批量的数据
            if i == self.len - 1 and not self.drop_reminder:
                batch_data = self.dataset[i * self.batch_size:]
            else:
                batch_data = self.dataset[i * self.batch_size: (i+1) * self.batch_size]

            src_idx, src_len, trg_idx = encode_and_pad(batch_data, self.max_len)
            # 将序列数据转换为tensor
            yield mindspore.Tensor(src_idx, mindspore.int32), \
                mindspore.Tensor(src_len, mindspore.int32), \
                mindspore.Tensor(trg_idx, mindspore.int32)

    def __len__(self):
        return self.len

In [11]:
train_iterator = Iterator(train_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=True)
valid_iterator = Iterator(valid_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)
test_iterator = Iterator(test_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)

## 模型解析

### 多头注意力

#### 自注意力 (Self-Attention)

相比注意力机制探究句子与句子之间的对应关系，自注意力关注一个句子中每个单词对于周边单词的重要性。它可以有效地识别句子中的代词指代，如在'`The animal` didn't cross the street because `it` was too tired'这句话中，'it'指代句中的'The animal'，所以自注意力会赋予'The'、'animal'更高的注意力分值。

![avatar](./assets/transformer_self_attention.png)

> 图片来源： [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/) by Jay Alammer

给定序列$X \in \mathbb{R}^{n \times d_{model}}$，序列长度为$n$，维度为$d_{model}$。在计算自注意力时，$Q = K = V = X$

随后计算自注意力分数：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_{model}}}\right)V$$

其中，序列中位置为$i$的词与位置为$j$的词之间的自注意力分数为：

$$\text{Attention}(Q, K, V)_{i,j} = \frac{\text{exp}\left(\frac{Q_iK_j^T}{\sqrt{d_{model}}}\right)}{\sum_{k=1}^{n}\text{exp}\left(\frac{Q_iK_k^T}{\sqrt{d_{model}}}\right)}V_j$$

#### 多头注意力 (Multi-Head Attention)

![avatar](./assets/transformer_multihead_attention.png)

> 图片来源：Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. Attention is all you need, 2017.

多头注意力是自注意力机制的扩展，它可以使模型通过不同的方式关注输入序列的不同部分，从而提升模型的训练效果。

相比于一次性计算维度为$d_{model}$的注意力分数，多头注意力通过对输入的embedding乘以不同的权重参数$W^{Q}$、$W^{K}$和$W^{V}$，将其映射到多个小维度空间中，我们称之为“头”（head），每个头部会并行计算自己的自注意力分数。

$$\text{head}_i = \text{Attention}(QW^Q_i, KW^K_i, VW^V_i) = \text{softmax}\left(\frac{Q_iK_i^T}{\sqrt{d_{k}}}\right)V_i$$

$W^Q_i \in \mathbb{R}^{d_{model}\times d_{k}}$、$W^K_i \in \mathbb{R}^{d_{model}\times d_{k}}$和$W^V_i \in \mathbb{R}^{d_{model}\times d_{v}}$为可学习的权重参数。一般为了平衡计算成本，我们会将头的query，key，value向量的维度缩小为原先的维度除以头的数量，$d_k = d_v = d_{model} / n_{head}$。

在获得多组自注意力分数后，我们将结果拼接到一起，得到多头注意力的最终输出。$W^O$为可学习的权重参数，用于将拼接后的多头注意力输出映射回原来的维度。

$$\text{MultiHead}(Q, K, V)=\text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$

简单来说，在多头注意力中，每个头部可以'解读'输入内容的不同方面，比如：捕捉全局依赖关系、关注特定语境下的词元、识别词和词之间的语法关系等。


### Transformer 架构

![avatar](./assets/transformer.png)

> 图片来源：Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. Attention is all you need, 2017.

与Seq2Seq类似，Transformer模型由编码器（Encoder）与解码器（Decoder）构成。编码器将源序列embedding中的信息进行学习、整合，并处理为一个抽象的连续表示，随后将其传入解码器。解码器结合编码器输出与先前生成的单词，逐步生成输出序列。

训练过程中，Transformer通过比较生成序列与真实目标序列的差距来更新网络参数，整体流程如下：

1. 嵌入：将输入序列映射至高维向量空间，即把每一个词转化为一个向量。
2. 位置编码：在内容编码的基础上添加位置信息。
3. 编码：将带有位置信息的输入序列放入编码器中，通过自注意力、前馈神经网络等网络层学习输入序列中的信息，输出上下文向量。
4. 解码：每次解码器通过编码器输出的上下文向量和目前已生成的序列生成下一个单词，并更新当前序列，直到生成完整的句子。
5. 损失计算与网络更新：模型计算生成序列与目标序列的偏差，并通过反向传播计算梯度，更新网络参数。

### 位置编码（Positional Encoding）

Transformer模型不包含CNN或者RNN，所以无法在模型中记录时序信息，这样会导致模型无法识别由顺序改变而产生的句子含义的改变，如“我爱我的小猫”和“我的小猫爱我”。

为了弥补这个缺陷，我们选择在输入数据中额外添加表示位置信息的位置编码，位置编码的计算方式如下：

![avatar](./assets/transformer_word_embedding.png)

如图，假设我们对$\text{<bos> Hello world ! <eos>}$进行编码，最终生成一个5x4的word embedding。其中每一行表示一个词元的向量，列数表示嵌入的维度。

对于索引为[pos, 2i]（所在列为偶数列）的元素，位置编码的计算方式如下：

$$PE_{(pos,2i)} = \sin\Bigg(\frac{pos}{10000^{2i/d_{\text{model}}}}\Bigg)$$

对于索引为[pos, 2i+1]（所在列为奇数列）的元素，位置编码的计算方式如下：

$$PE_{(pos,2i+1)} = \cos\Bigg(\frac{pos}{10000^{2i/d_{\text{model}}}}\Bigg)$$

这样可以保证每一个位置的词元对应的位置编码都有所不同，位置编码另一个好处是，由于使用了$\sin$和$\cos$函数，它可以表示词元之间的相对信息，即位置编码记录的是两个词之间的相对距离，从而使得它们之间的逻辑关系不会被这组词在序列中的整体位置影响。如：

- “我不喜欢吃榴莲，因为它太臭了。”
- “妈妈今天买了榴莲，可惜我不喜欢吃榴莲。”

“我不喜欢吃榴莲”这个片段分别出现在了两个句子的句首和句尾，但均表示了“我”对“榴莲”的负面情绪。

In [12]:
import mindspore.ops as ops
import mindspore.nn as nn


class PositionalEncoding(nn.Cell):
    """位置编码"""

    def __init__(self, embed_dim, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=1 - dropout)

        # 位置信息
        # shape = [batch size = 1, max len, embed dim]
        self.pos = ops.fill(compute_dtype, (1, max_len, embed_dim), 0)
        angle = ops.arange(end=max_len, dtype=compute_dtype).reshape(
            -1, 1) / ops.pow(
                10000,
                ops.arange(end=embed_dim, step=2, dtype=compute_dtype) /
                embed_dim)
        self.pos[:, :, 0::2] = ops.sin(angle)
        self.pos[:, :, 1::2] = ops.cos(angle)

    def construct(self, x):
        # 将位置编码截取至x同等大小
        # shape = [batch size = 1, seq len, embed dim]
        x = x + self.pos[:, :x.shape[1], :]
        return self.dropout(x)

### 编码器（Encoder）

![avatar](./assets/transformer_encoder.png)

Transformer的Encoder负责处理输入的源序列，并将输入信息整合为一系列的上下文向量（context vector）输出。它是由多个相同结构堆叠而成，每个结构中包含多头自注意力机制和前馈神经网络（Feed-Forward Network，FFN）两个子层，子层之间通过残差结构和Add&Norm层进行连接，帮助稳定数据传输，加速模型收敛。

多头自注意力我们在上文已经进行了详细的介绍，接下来重点讲解前馈神经网络和Add&Norm。

#### Add & Norm

$\text{Add\&Norm}(x) = \text{LayerNorm}(x + \text{Sublayer}(x))$

- Add：残差连接

- Norm：Layer Norm

![avatar](./assets/transformer_layernorm_vs_batchnorm.png)

> 图片来源：Yuxin Wu and Kaiming He. Group normalization. In Proceedings of the European Conference on Computer Vision (ECCV), September 2018

如图，在语言模型中，H、C、N一般对应embedding size，sequence length和batch size。Batch Norm是对每一个特征进行归一化，但在语句长度（C）不固定的情况下计算会非常不方便；于是我们选择对每个样本（N）中的元素进行归一化，即Layer Norm。


### 前馈神经网络（FFN）

前馈神经网络是Encoder结构中的第二个子层，用来对输入的每个词元进行非线性变换，从而使模型更好地捕捉输入信息。

$$\mathrm{FFN}(x) = \mathrm{ReLU}(xW_1 + b_1)W_2 + b_2$$

非线性变化主要分为三步：

1. $z_1 = xW_1 + b_1$：线性变化，将输入$x$的维度从$d_{model}$变为$d_{ff}$。
2. $z_2 = \mathrm{ReLU}(z_1)$：非线性变化
3. $\text{output} = z_2W_2 + b_2$：线性变化，将维度从$d_{ff}$变回$d_{model}$。

### 解码器（Decoder）

![avatar](./assets/transformer_decoder.png)

解码器将编码器输出的上下文序列转换为目标序列的预测结果$\hat{Y}$，该输出将在模型训练中与真实目标输出$Y$进行比较，计算损失。

解码器的结构与编码器相似，也是多个相同结构的堆叠，但相比编码器，解码器每个结构中有两层多头注意力机制。第一层是计算目标序列的注意力分数的*掩码多头注意力*，第二层是用于计算上下文序列与目标序列对应关系，其中Decoder掩码多头注意力的输出作为query，Encoder的输出（上下文序列）作为key和value。

在N个解码器结构之后，输出会经过一个全连接层，变为对目标序列的预测结果。

#### 带掩码的多头注意力 （Masked Multi-Head Attention）

前序在处理数据时，我们为了统一长度，使用\<pad\>占位符补齐了一些稍短的文本，这些\<pad\>占位符没有任何意义，不应该参与注意力分数计算中。为此我们在多头注意力中加入了内容掩码（padding mask），即识别输入序列中的\<pad\>占位符，保证计算时这些位置对应的注意力分数为0。

In [13]:
import mindspore.ops as ops


def get_attn_pad_mask(seq_q, seq_k, pad_idx, broadcast=False):
    """注意力掩码：识别序列中的<pad>占位符

    Args:
        seq_q (Tensor): query序列，shape = [batch size, query len]
        seq_k (Tensor): key序列，shape = [batch size, key len]
        pad_idx (Tensor): key序列<pad>占位符对应的数字索引
        broadcast (bool): 是否需要广播机制。默认：False
            如True，返回shape = [batch size, query len, key len]；如False，返回shape = [batch size, key len]
    """
    batch_size, len_q = seq_q.shape
    batch_size, len_k = seq_k.shape

    # 如果序列中元素对应<pad>占位符，则该位置在mask中对应元素为True
    # shape = []
    pad_attn_mask = ops.equal(seq_k, pad_idx)

    if broadcast:
        # 增加额外的维度
        # shape = [batch size, 1, key len]
        pad_attn_mask = pad_attn_mask.expand_dims(1)
        # 将掩码广播到[batch size, query len, key len]
        pad_attn_mask = ops.broadcast_to(pad_attn_mask,
                                         (batch_size, len_q, len_k))

    return pad_attn_mask

而在预测中，我们通过目前现阶段已有的序列预测它的下一个词语是什么，相应地在训练中，t时刻的模型只能“观察”直到t-1时刻的所有词元，后续的词语不应该一并输入Decoder中。

为了保证在t时刻，只有t-1个词元作为输入参与多头注意力分数的计算，我们需要在第一个多头注意力中额外增加一个时间掩码，使目标序列中的词随时间发展逐个被暴露出来。

该注意力掩码可通过三角矩阵`mindspore.ops.triu`实现，对角线以上的词元表示为不参与注意力计算的词元，标记为1。

$$\begin{matrix}
0 & 1 & 1 & 1 & 1\\
0 & 0 & 1 & 1 & 1\\
0 & 0 & 0 & 1 & 1\\
0 & 0 & 0 & 0 & 1\\
0 & 0 & 0 & 0 & 0\\
\end{matrix}$$

该掩码一般被称作subsequent mask。

In [14]:
import mindspore.ops as ops


def get_attn_subsequent_mask(seq_q, seq_k):
    """生成时间掩码，使decoder在第t时刻只能看到序列的前t-1个元素
    
    Args:
        seq_q (Tensor): query序列，shape = [batch size, query len]
        seq_k (Tensor): key序列，shape = [batch size, key len]
        pad_idx (Tensor): key序列<pad>占位符对应的数字索引
    """
    _, len_q = seq_q.shape
    _, len_k = seq_k.shape
    # 生成三角矩阵
    # shape = [query len, key len]
    ones = ops.ones((len_q, len_k), mindspore.float32)
    subsequent_mask = ones.triu(diagonal=1)
    # 在第0维增加额外维度
    # shape = [1, query len, key len]
    subsequent_mask = subsequent_mask.expand_dims(0)
    return subsequent_mask

最后，将subsequent mask和padding mask合并为一个整体的掩码，确保模型既不会注意到t时刻以后的词元，也不会关注为\<pad\>的词元。

In [15]:
import mindspore.ops as ops


def get_enc_dec_mask(src, trg, src_pad_idx, trg_pad_idx):
    """获取encoder与decoder中的掩码"""
    # encoder 自注意力<pad>占位符掩码
    # shape = [batch size, src len]
    enc_self_attn_mask = get_attn_pad_mask(src, src, src_pad_idx)

    # decoder 带有时间限制的自注意力掩码
    # 分别计算<pad>占位符掩码与时间掩码，并合并
    # shape = [batch size, trg len, trg len]
    dec_self_attn_pad_mask = get_attn_pad_mask(trg,
                                               trg,
                                               trg_pad_idx,
                                               broadcast=True)
    dec_self_attn_subsequent_pad_mask = get_attn_subsequent_mask(trg, trg)
    dec_self_attn_mask = dec_self_attn_pad_mask + dec_self_attn_subsequent_pad_mask
    dec_self_attn_mask = ops.gt((dec_self_attn_mask), 0)

    # decoder 注意力<pad>占位符掩码
    # shape = [batch size, src len]
    dec_enc_attn_mask = get_attn_pad_mask(trg, src, src_pad_idx)

    return enc_self_attn_mask, dec_self_attn_mask, dec_enc_attn_mask

## 模型构建

在Transformer层前后补充embedding，位置编码，全连接层。

In [16]:
class TransformerNetwork(nn.Cell):

    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers,
                 dim_feedforward, max_len, src_vocab_size, trg_vocab_size):
        super().__init__()

        # 多头注意力头的数量
        self.nhead = nhead
        # 位置掩码
        self.positional_encoding = PositionalEncoding(d_model,
                                                      max_len=max_len + 1)
        # 缩放比例
        self.content_scale = ops.sqrt(
            mindspore.Tensor([d_model], mindspore.float32))
        # 源序列embedding
        self.src_content_emb = nn.Embedding(src_vocab_size, d_model)
        # 目标序列embedding
        self.trg_content_emb = nn.Embedding(trg_vocab_size, d_model)
        # transformer层
        self.transformer = nn.Transformer(d_model,
                                          nhead,
                                          num_encoder_layers,
                                          num_decoder_layers,
                                          dim_feedforward,
                                          batch_first=True)
        # 全连接层
        self.projection = nn.Dense(d_model, trg_vocab_size, has_bias=False)

    def construct(self, src, trg, src_pad_idx, trg_pad_idx):
        # 计算源序列与目标序列的embedding，并加入位置编码
        # 为平衡原embedding和位置编码的数值，将原embedding以适当比例放大
        # src shape = [batch size, src len, model dim]
        # trg shape = [batch size, trg len, model dim]
        src_emb = self.positional_encoding(
            self.src_content_emb(src) * self.content_scale)
        trg_emb = self.positional_encoding(
            self.trg_content_emb(trg) * self.content_scale)

        # encoder与decoder中的掩码
        # 注意将dec_self_attn_mask的shape变为[batch size * head num, trg len, trg len]
        enc_self_attn_mask, dec_self_attn_mask, dec_enc_attn_mask = get_enc_dec_mask(
            src, trg, src_pad_idx, trg_pad_idx)
        dec_self_attn_mask = ops.tile(dec_self_attn_mask, (self.nhead, 1, 1))

        # transformer输出
        output = self.transformer(src_emb,
                                  trg_emb,
                                  src_key_padding_mask=enc_self_attn_mask,
                                  memory_key_padding_mask=dec_enc_attn_mask,
                                  tgt_mask=dec_self_attn_mask)
        
        # logits计算
        # shape = [batch size, trg len, trg vocab size]
        dec_logits = self.projection(output)

        # shape = [batch size * trg len, trg vocab size]
        dec_logits = dec_logits.view((-1, dec_logits.shape[-1]))

        return dec_logits.astype(compute_dtype)

## 梯度裁剪

在模型训练中，可能会出现权重更新过大的情况。这会导致数值上溢或者下溢，最终造成梯度爆炸（gradient explosion）。为解决这个问题，我们需要在反向传播计算梯度之后，使用梯度裁剪(gradient clipping)，再将裁剪后的梯度传入优化器进行网络更新。

In [17]:
class ClipByNorm(nn.Cell):
    """梯度裁剪"""
    
    def __init__(self, clip_norm):
        super().__init__()

        self.hyper_map = ops.HyperMap()
        self.clip_norm = clip_norm
    
    @staticmethod
    def _clip_by_norm(clip_norm, t, axis=None):
        # 计算L2-norm
        t2 =  t * t
        l2sum = t2.sum(axis=axis, keepdims=True)
        pred = l2sum > 0
        # 将加和中等于0的元素替换为1，避免后续出现NaN
        l2sum_safe = ops.select(pred, l2sum, ops.ones_like(l2sum))
        l2norm = ops.select(pred, ops.sqrt(l2sum_safe), l2sum)
        # 比较L2-norm和clip_norm，如L2-norm超过阈值，进行裁剪
        # output(x) = (x * clip_norm)/max(|x|, clip_norm)
        intermediate = t * clip_norm
        cond = l2norm > clip_norm
        t_clip = intermediate / ops.select(cond, l2norm, clip_norm)

        return t_clip
    
    def forward(self, grads, axis=None):

        grads = self.hyper_map(ops.partial(self._clip_by_norm, self.clip_norm), grads) 

        return grads
        

## 模型训练

### 模型初始化

模型、损失函数、优化器初始化。

In [18]:
d_model = 512  # Embedding层维度
n_head = 8  # 多头感知机中头的数量
n_layer = 6  # 编码器和解码器的层数
d_ff = 2048  # 前馈神经网络维度
max_len = 32  # 序列最大长度

compute_dtype = mindspore.float32

positional_encoding = PositionalEncoding(d_model, max_len=max_len + 1)  # 位置掩码
src_content_emb = nn.Embedding(len(de_vocab), d_model)  # 源序列embedding
trg_content_emb = nn.Embedding(len(en_vocab), d_model)  # 目标序列embedding
src_pad_idx = de_vocab.pad_idx  # 源序列中<pad>占位符索引
trg_pad_idx = en_vocab.pad_idx  # 目标序列中<pad>占位符索引
src_vocab_size = len(de_vocab)
trg_vocab_size = len(en_vocab)

model = TransformerNetwork(d_model, n_head, n_layer, n_layer, d_ff, max_len,
                           src_vocab_size, trg_vocab_size)
loss_fn = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
opt = nn.Adam(model.trainable_params(), learning_rate=0.0001)
clip_by_norm = ClipByNorm(clip_norm=1.0)

### 定义模型训练过程

在训练过程中，表示句子结尾的\<eos\>占位符应是被模型预测出来，而不是作为模型的输入，所以在处理Decoder的输入时，我们需要移除目标序列最末的\<eos\>占位符。

$$\text{trg} = [\text{<bos>}, x_1, x_2, ..., x_n, \text{<eos>}]$$
$$\text{trg[:-1]} = [\text{<bos>}, x_1, x_2, ..., x_n]$$

其中，$x_i$代表目标序列中第i个表示实际内容的词元。

我们期望最终的输出包含表示句末的\<eos\>，不包含表示句首的\<bos\>，所以在计算损失时，需要同样去除的目标序列的句首\<bos\>占位符，再进行比较。

$$\text{output} = [y_1, y_2, ..., y_n, \text{<eos>}]$$
$$\text{trg[1:]} = [x_1, x_2, ..., x_n, \text{<bos>}]$$

其中，$y_i$表示预测的第i个实际内容词元。

In [19]:
def forward_fn(src, trg, src_pad_idx, trg_pad_idx):
    """前向网络"""
    # output shape = [batch size * (trg len - 1), trg vocab size]
    # trg[:, 1:].view(-1) shape = [batch size * (trg len - 1)]
    output = model(src, trg[:, :-1], src_pad_idx, trg_pad_idx)
    loss = loss_fn(output, trg[:, 1:].view(-1))

    return loss


# 反向传播计算梯度
grad_fn = mindspore.value_and_grad(forward_fn, None, opt.parameters)


def train_step(src, trg, src_pad_idx, trg_pad_idx):
    """单步训练"""
    # 计算损失与梯度
    loss, grads = grad_fn(src, trg, src_pad_idx, trg_pad_idx)
    # 梯度裁剪
    grads = clip_by_norm(grads)
    # 更新网络权重
    opt(grads)
    return loss


def train(iterator, epoch=0):
    """模型训练"""
    model.set_train(True)
    num_batches = len(iterator)
    total_loss = 0  # 所有batch训练loss的累加
    total_steps = 0  # 训练步数

    with tqdm(total=num_batches) as t:
        t.set_description(f'Epoch: {epoch}')
        for src, src_len, trg in iterator():
            # 计算当前batch数据的loss
            loss = train_step(src, trg, src_pad_idx, trg_pad_idx)
            total_loss += loss.asnumpy()
            total_steps += 1
            # 当前的平均loss
            curr_loss = total_loss / total_steps
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)

    return total_loss / total_steps


def evaluate(iterator):
    """模型验证"""
    model.set_train(False)
    num_batches = len(iterator)
    total_loss = 0  # 所有batch训练loss的累加
    total_steps = 0  # 训练步数

    with tqdm(total=num_batches) as t:
        for src, src_len, trg in iterator():
            # 当前batch数据的loss
            loss = forward_fn(src, trg, src_pad_idx, trg_pad_idx)
            total_loss += loss.asnumpy()
            total_steps += 1
            # 当前的平均loss
            curr_loss = total_loss / total_steps
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)

    return total_loss / total_steps

模型训练，训练途中使用验证数据集进行验证评估，并保存效果最好的模型。

In [22]:
from mindspore import save_checkpoint

num_epochs = 1 # 训练迭代数
best_valid_loss = float('inf') # 当前最佳验证损失
ckpt_file_name = os.path.join(cache_dir, 'transformer.ckpt') # 模型保存路径


for i in range(num_epochs):
    # 模型训练，网络权重更新
    train_loss = train(train_iterator, i)
    # 网络权重更新后对模型进行验证
    valid_loss = evaluate(valid_iterator)
    
    # 保存当前效果最好的模型
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        save_checkpoint(model, ckpt_file_name)

100%|██████████| 8/8 [00:02<00:00,  3.44it/s, loss=1.22]


## 模型推理

对输入的单个句子进行文本翻译，步骤如下：

1. 数据预处理：将该句子进行分词，统一大小写，补充句首句末占位符，统一序列长度，将词元转换为数字索引，最后将数字索引序列转换为tensor；
2. 创建列表`trg_indexes`，用以储存生成的英语词元，\<bos\>作为第一个输入；
3. 开始预测，将每次预测的结果放入列表`trg_indexes`，并将该列表作为输入，预测下一个词元；
4. 重复上述步骤，直至输出\<eos\>或达到句子长度限制；
5. 去除首位占位符，参照词典，将预测结果（数字索引）转换为英文单词。

In [ ]:
def translate_sentence(sentence, de_vocab, en_vocab, model, max_len=32):
    """给定德语句子，返回英文翻译"""
    model.set_train(False)
    # 对输入句子进行分词
    if isinstance(sentence, str):
        spacy_lang = spacy.load('de')
        tokens = [token.text.lower() for token in spacy_lang(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    
    # 补充起始、终止占位符，统一序列长度
    if len(tokens) > max_len - 2:
        src_len = max_len
        tokens = ['<bos>'] + tokens[:max_len - 2] + ['<eos>']
    else:
        src_len = len(tokens) + 2
        tokens = ['<bos>'] + tokens + ['<eos>'] + ['<pad>'] * (max_len - src_len)
        
    # 将德语单词转化为数字索引
    src = de_vocab.encode(tokens)
    src = mindspore.Tensor(src, mindspore.int32).expand_dims(0)
    print(src.shape)
    src_len = mindspore.Tensor([src_len], mindspore.int32)
    trg_indexes = [en_vocab.bos_idx]
    trg = mindspore.Tensor(trg_indexes, mindspore.int32).expand_dims(0)
    print(f'trg shape: {trg.shape}')
    
    for i in range(max_len):
        outputs = model(src, trg, src_pad_idx, trg_pad_idx)
        # print(f'outputs suqeeze shape: {outputs.squeeze().shape}')
        # print(f'output shape: {outputs.shape}')
        # print(f'outputs argmax shape: {outputs.argmax(1).shape}')
        # print(f'outputs: {outputs.asnumpy()}')
        # print(f'outputs argmax: {outputs.argmax(1).asnumpy()}')
        pred = int(outputs.argmax(1).asnumpy()[-1])
        print(f'pred: {pred}')
        
        if pred == en_vocab.eos_idx:
            break
        
        trg_indexes.append(pred) 
        trg = mindspore.Tensor(trg_indexes, mindspore.int32).expand_dims(0)

    trg_tokens = en_vocab.decode(trg_indexes)
    
    return trg_tokens

In [ ]:
from mindspore import load_checkpoint, load_param_into_net

# 加载之前训练好的模型
param_dict = load_checkpoint(ckpt_file_name)
load_param_into_net(model, param_dict)

# 以测试数据集中的第一组语句为例，进行测试
example_idx = 0

src = test_dataset[example_idx][0]
trg = test_dataset[example_idx][1]

print(f'src = {src}')
print(f'trg = {trg}')

查看预测结果

In [ ]:
translation = translate_sentence(src, de_vocab, en_vocab, model)

print(f'predicted trg = {translation}')